<a href="https://colab.research.google.com/github/adithya-prabhu-22/deep_learning_algos/blob/main/imdb_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from keras.models import *
from keras.layers import *
from keras.datasets import imdb
from keras import Sequential

In [67]:
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=10000)

In [31]:
x_train
#x_train,x_test and train are integer encoded we need to convert this into embeddings

array([list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]),
       list([1, 194, 1

In [74]:
# x_train=pad_sequences(x_train,maxlen=50)
# x_test=pad_sequences(x_test,maxlen=50)

# The padding step will be done directly before training to ensure the correct data is used.
# Original comment: converting all doc into len of 50 words larger docs are trimmed and smaller than 50 filled with 0 in backside upto 150

In [42]:
x_train.shape

(25000, 50)

In [51]:
from keras.utils import *

Let's redefine the model using the Keras Functional API to explicitly define the input layer. This can sometimes help resolve subtle issues with input shapes and model building in TensorFlow.

In [69]:
from keras.models import Model
from keras.layers import Input, Embedding, SimpleRNN, Dense

# Define the input layer
# The shape matches the padded sequences (length 50)
input_layer = Input(shape=(50,))

# Add the Embedding layer, connecting it to the input layer
embedding_layer = Embedding(input_dim=10000, output_dim=32, input_length=50)(input_layer)

# Add the SimpleRNN layer, connecting it to the embedding layer
# return_sequences=False is used for many-to-one classification
rnn_layer = SimpleRNN(32, return_sequences=False)(embedding_layer)

# Add the Dense output layer, connecting it to the RNN layer
output_layer = Dense(1, activation='sigmoid')(rnn_layer)

# Create the model by specifying the input and output layers
model_functional = Model(inputs=input_layer, outputs=output_layer)

# Compile the model (same as before)
model_functional.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model_functional.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_8 (Embedding)         │ (None, 50, 32)         │       320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_11 (SimpleRNN)       │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 322,113 (1.23 MB)

 Trainable params: 322,113 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

Now let's try training this functional API model.

In [73]:
from keras.utils import pad_sequences
import numpy as np

# Pad the sequences again just before training to ensure correctness
# Assuming maxlen=50 as used previously
x_train_padded = pad_sequences(x_train, maxlen=50)
x_test_padded = pad_sequences(x_test, maxlen=50)

# Ensure y_train is a numpy array with float32 dtype
y_train_np = np.array(y_train).astype('float32')

# Train the functional API model
# Use the padded data for training
history = model_functional.fit(x_train_padded, y_train_np, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.6569 - loss: 0.5855 - val_accuracy: 0.8020 - val_loss: 0.4422
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.8691 - loss: 0.3144 - val_accuracy: 0.7966 - val_loss: 0.4674
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.9451 - loss: 0.1596 - val_accuracy: 0.7844 - val_loss: 0.5740
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.9857 - loss: 0.0559 - val_accuracy: 0.7738 - val_loss: 0.7509
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 14ms/step - accuracy: 0.9949 - loss: 0.0217 - val_accuracy: 0.7620 - val_loss: 0.8654
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.9983 - loss: 0.0091 - val_accuracy: 0.7648 - val_loss: 0.9709
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.9989 - loss: 0.0059 - val_accuracy: 0.7430 - val_loss: 1.1725
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - accuracy: 0.9900 - loss: 0.0272 - val_

In [76]:
# Use the trained functional model for prediction
y_pred = model_functional.predict(x_test_padded)

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step


In [77]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(y_test, y_pred.round())
#

0.7594